In [ ]:
import pandas as pd
import numpy as np

def mudaAnoUgestora(ano):
    if ano >= 2009 and ano <= 2012:
        return 2009
    elif ano >= 2013 and ano <= 2016:
        return 2013

def nomeUgestora(gestora):
    return gestora[24:].upper()

def upperPrefeito(nome):
    return nome.upper()

def anoAposEleicao(ano):
    if ano == 2008:
        return 2009
    elif ano == 2012:
        return 2013

path_ges, path_jul  = r'~/Downloads/codigo_ugestora.csv', r'~/Downloads/contas_julgadas_irregulares_TRE_2016.csv'
ud_gestoras, contas_jul_irr = pd.read_csv(path_ges), pd.read_csv(path_jul)

#Filtro por ano entre 2008 e 2016
ud_gestoras = ud_gestoras[ud_gestoras['dt_Ano']>2008]

#Solucao do ano
vfunc = np.vectorize(mudaAnoUgestora)
anos = vfunc(ud_gestoras['dt_Ano'].values)
ud_gestoras['dt_Ano'] = anos

#Solucao do nome da gestora
vfunc = np.vectorize(nomeUgestora)
ud_gestoras_pref = ud_gestoras[ud_gestoras['de_Ugestora'].str.contains("Prefeitura")]
municipios = vfunc(ud_gestoras_pref['de_Ugestora'].values)
ud_gestoras_pref['de_Ugestora'] = municipios
ud_gestoras_pref = ud_gestoras_pref[['cd_Ugestora','dt_Ano','de_Ugestora']]

path_cand_2008, path_cand_2012 = r'~/Downloads/candidate_local_2008-1.csv', r'~/Downloads/candidate_local_2012-1.csv'

#para 2008
cand_2008 = pd.read_csv(path_cand_2008)
cand_2008_PB = cand_2008[(cand_2008['SIGLA_UF'] == 'PB')]
cand_2008_PB = cand_2008_PB[(cand_2008_PB['CODIGO_CARGO'] == 11)]
cand_2008_PB_ELEIT = cand_2008_PB[(cand_2008_PB['DESC_SIT_TOT_TURNO'] == 'ELEITO')]
cand_2008_PB_ELEIT = cand_2008_PB_ELEIT[['ANO_ELEICAO','DESCRICAO_UE', 'NOME_CANDIDATO']]

#para 2012
cand_2012 = pd.read_csv(path_cand_2012)
cand_2012_PB = cand_2012[(cand_2012['SIGLA_UF'] == 'PB')]
cand_2012_PB = cand_2012_PB[(cand_2012_PB['CODIGO_CARGO'] == 11)]
cand_2012_PB_ELEIT = cand_2012_PB[(cand_2012_PB['DESC_SIT_TOT_TURNO'] == 'ELEITO')]
cand_2012_PB_ELEIT = cand_2012_PB_ELEIT[['ANO_ELEICAO','DESCRICAO_UE', 'NOME_CANDIDATO']]

#concatenação de 2008 e 2012
eleicoes2008_2012 = [cand_2008_PB_ELEIT,cand_2012_PB_ELEIT]
result = pd.concat(eleicoes2008_2012)

vfunc = np.vectorize(anoAposEleicao)
anos = vfunc(result['ANO_ELEICAO'].values)
result['ANO_ELEICAO'] = anos

result.rename(columns={'DESCRICAO_UE': 'de_Ugestora', 'ANO_ELEICAO':'dt_Ano'}, inplace=True)
final = pd.merge(ud_gestoras_pref, result, on=['de_Ugestora', 'dt_Ano'])
final = final.drop_duplicates()
final.rename(columns={'NOME_CANDIDATO': 'RESPONSÁVEL'}, inplace=True)


#Sobre a tabela do TCE
contas_jul_irr.rename(columns={'JURISDICIONADO': 'de_Ugestora'}, inplace=True)

vfunc = np.vectorize(upperPrefeito)
responsaveis = vfunc(contas_jul_irr['RESPONSÁVEL'].values)
contas_jul_irr['RESPONSÁVEL'] = responsaveis

vfunc = np.vectorize(nomeUgestora)
contas_jul = contas_jul_irr[contas_jul_irr['de_Ugestora'].str.contains("Prefeitura")]
municipios = vfunc(contas_jul['de_Ugestora'].values)
contas_jul['de_Ugestora'] = municipios

all_final = pd.merge(contas_jul, final, on=['de_Ugestora','RESPONSÁVEL'])

all_final.to_csv('~/Desktop/TRE_Sagres.csv',sep=',',encoding='utf-8', index=False)